# Classificação _20 New Groups_

Primeiramente precisamos definir quais são os nosso dados. 
Os dados são uma coleção de aproximadamente 20000 documentos, particionados quase que igualmente entre 20 diferentes classes. Esses documentos são basicamente textos (linguagem natural) de emails correspondente à certos tópicos (sua classe).

Os 20 grupos distintos de documentos são: 

|Classe||
|:----:|:---:|
|comp.graphics|comp.os.ms-windows.misc|
|comp.sys.ibm.pc.hardware|comp.sys.mac.hardware|
|comp.windows.x|
|rec.autos|rec.motorcycles|
|rec.sport.baseball|rec.sport.hockey|	
|sci.crypt|sci.electronics|
|sci.med|sci.space|
|misc.forsale|
|talk.politics.misc|talk.politics.guns|
|talk.politics.mideast|	talk.religion.misc|
|alt.atheism|soc.religion.christian|

Vale resaltar que os documentos estão organizados em diretórios, cada qual nomeado correspondentemente com sua classe.

[Download do conjunto de dados 18828\*](http://qwone.com/~jason/20Newsgroups/20news-18828.tar.gz)       
[Referência dos dados](http://qwone.com/~jason/20Newsgroups/)

_* Os dados utilizados serão referentes ao conjunto 18828, que remove duplicatas e cabeçalhos desnecessários._

## Importação dos dados
Para a importação dos dados, vamos utilizar a biblioteca do _scikit-learn_, [**load_files**](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_files.html). Essa função utiliza dos nomes das sub-pastas, que estão contidas no diretório alvo para cadastrar as múltiplas classes, sendo assim, ao final da importação teremos tanto nosso vetor X quanto nosso vetor y.

In [38]:
from time import time
from sklearn.datasets import load_files

t0 = time()
dataset = load_files(container_path='dataset', shuffle=True)
print('Import Time: %0.3fs' % (time() - t0))
X, y = dataset.data, dataset.target
print('X size: %d\ny size: %d' % (len(X), len(y)))

Import Time: 3.462s
X size: 18828
y size: 18828


### Pre-processamento dos dados
Após a importação dos nossos vetores, podemos partir para o tratamento dos nossos dados.   
Como estamos trabalhando com textos, precisamos transformar isso em algo compreensível para o computador e para nossos algoritmos de classificação.   
Dessa forma, precisamos transformar os dados textuais em números, ou melhor uma matriz numérica. Para tanto ulizaremos de uma gama de técnicas de processamento de linguagem natural.

### Biblioteca de processamento de linguagem natural
Para executar algumas tarefas de processamento de linguagem natural iremos utilizar da biblioteca [**NLTK**](http://www.nltk.org), _Natural Language Toolkit_.   

#### _Stop Words_
Essa biblioteca nos auxiliará a realizar tarefas como a retirada de palavras de parada (ou palavras vazias), que podem ser consideradas irrelevantes ao contexto da fala e escrita. Podemos dizer que essas palavras são as mais comuns da língua ou palavras funcionais (_function words_), porém não existe uma lista universal de _stop words_, sendo que essas variam de contexto para contexto.   
Para o nosso contexto, a lista disponível na biblioteca NLTK será o suficiente para o processamento inicial.

#### Tokenização de palavras
Também é necessário transformar nosso texto em '_tokens_', que é nosso documento quebrado em palavras, ou _tokens_. _Token_ é um pedaço do todo, então uma palavra é um _token_ em uma frase, e uma frase é um _token_ em um parágrafo. Para nosso problema transfomaremos nosso documento (documento sendo um conjunto de frases ou paragrafos) em palavras, formando um vetor.

#### _Stemming_
_Stemming_ é o processo de chegar a raiz de uma palavra, via cortes. Sendo assim, com o uso de regras básicas, qualquer _token_ pode ser lapidado à sua raiz, dessa forma, _stemming_ é um processo baseado em regras no qual queremos juntar toda variação diferente do _token_.

#### _Lemmatization_
_Lemmatization_ é uma maneira metódica de converter toda forma gramatical ou inflacionada de um termo à sua raiz. _Lemmatization_ usa o contexto e parte de fala (_part of speech_) para determinar a forma inflacionada de uma palavra e aplica diferentes regras de normalização para cada parte de fala para alcançar a forma raiz do temo (_lemma_).

Para biblioteca NLTK é utilizado a base de dados _WordNet_ para atingir a raiz de cada _token_.

##### Spacy (Outra biblioteca de NLP)
Vale a pena citarmos outra biblioteca de processamento de dados de linguagem natural, a [**Spacy**](https://spacy.io/docs/usage/), que assim como a NLTK se especializa em tratamento de dados textuais.   
Apesar dessa biblioteca ter uma usabilidade muito boa (não é necessario vários _imports_ ou linhas de código para atingir um objetivo), optamos por utilizar a NLTK pela sua maior base na comunidade (16 anos no mercado) e seu tempo de execução para métodos como _Lemmatization_, que é muito menor que o do **Spacy**.

In [7]:
import re
import nltk # Biblioteca de processamento de linguagem natural
# Execute essa linha abaixo caso ocorra um erro de dependência
# nltk.download('punkt') # Necessário para tokenizar (sent_tokenize)
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer

def pre_process_text(text, lemmatize):
    if not isinstance(text, str):
        text = text.decode('ISO-8859-1')
    
    text = re.sub('[^a-zA-Z]', ' ', text) # Retirar caracteres especiais e digitos
    text = text.lower() # Tudo para caixa baixa
    text = text.split() # Retirar espaços exessivos
    text = ' '.join(text)
    # print('\tTexto limpo.\n', text)

    # Tokenizar
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    # print('\tTransformando em tokens.\n', tokens)

    # Remover as stopword
    stop = stopwords.words('english')
    tokens = [token for token in tokens if token not in set(stop)]
    # print('\tRetirando stopwords.\n', tokens)
    
    # Tirar palavras menores que 3 caracteres
    tokens = [token for token in tokens if len(token) >= 2]
    # print('\tRetirando palavras menores que 2.\n', tokens)
    

    if lemmatize:
        # Lemmatize
        lemmatizer = WordNetLemmatizer()
        tokens = [lemmatizer.lemmatize(word) for word in tokens]
        # print('\tLemmatizing.\n', tokens)
    else:
        stemmer = PorterStemmer()
        tokens = [stemmer.stem(word) for word in tokens]
    
    # Texto pre processado
    tokens = ' '.join(tokens)
    
    return tokens

In [39]:
t0 = time()
X = [pre_process_text(doc, True) for doc in X]
print('Cleaning time: %0.3fs' % (time() - t0))

Cleaning time: 75.818s


### Extração de Características (_Features Extraction_)
Agora que temos nossos dados limpos, podemos transforma-los em uma representação numérica, ou melhor em uma matriz.

#### _Bag of Words_ (Punhado de palavras)
A forma mais comum de alcançar esse objetivo é por meio do _Bag of Words_ que "aprende" um vocabulário a partir do nosso _corpus_ (conjunto de documentos) então modela cada documento a partir a contagem da ocorrencia de cada palavra.   
Dessa forma descartamos a maioria da estrutura de um texto, como paragrafos, frases e formatação, apenas contando quão frequente é uma palavra em cada documento. Sendo assim, essa técnica levou a imagem de um saco, ou punhado de palavras.   
Essa técnica é implementada pela função **_CountVectorizer_** da biblioteca _sklearn_.

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_bow = CountVectorizer(max_features=100000) # Retiramos cerca de 2000 features
t0 = time()
X_bow = vectorizer_bow.fit_transform(X)
print('Bag of words time: %0.3fs' % (time() - t0))

Bag of words time: 9.151s


#### TF-IDF (Term Frequency - Inverse Document Frequency)
_TF-IDF_ é uma boa medida estatística para refletir a relevância de um termo para o documento em uma coleção de documentos. Esse método calcula um peso para cada termo com base em quão frequente ele em um documento, mas não em vários doumentos no _corpus_. Sendo assim, se uma palavra aparece muito frequentemente em um documento especifico, mas não em vários documentos, é provável que esse termo é importante para aquele contexto daquele documento.   
Para calcular esse peso o algoritmo utiliza das fórmulas abaixo:
$$ tf = \frac{Número\ de\ vezes\ que\ um\ termo\ aparece\ no\ documento}{Total\ de\ termos\ no \ documento}$$

$$df = \frac{d(número\ de\ documentos\ contendo\ um\ termo\ específico)}{D (tamanho\ do\ corpus)}$$

Normalmente D > d e $\log\frac{d}{D}$ resulta em um valor negativo. Portanto, tomamos o inverso dessa fração e essencialmente comprimimos a escala dos calores para que quantidades muito altas ou muito baixas sejam fácilmente comparáveis.

$$ idf = \log\big(\frac{Total\ de\ documentos}{Número\ de\ documentos\ contendo\ um\ termo\ específico}\big)$$

$$tfidf = \text{tf} \log\big(\frac{Número\ total\ de\ documentos + 1}{Número\ de\ documentos\ contendo\ um\ termo\ específico + 1}\big) + 1$$

Para executar o _TF-IDF_ foi utilizada a função da biblioteca _sklearn_ [**_TfidfVectorizer_**](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer).

##### Utilizando da função TfidfVectorizer
Para a função da biblioteca do _sklearn_ existe a possibilidade de utilizarmos N-Grams, que é a sequencialização de tokens, formando novos padrões, onde o contexto pode gerar mais informação.   
Com isso podemos ter um entendimento maior de alguns assuntos, sendo assim, essa funcionalidade foi utilizada no treinamento da nossa aplicação.



In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer_tfidf = TfidfVectorizer(min_df=1,
                                   max_df=0.7, # Retiramos palavras com alta recorrência em documentos, como "From" e "Subject"
                                   ngram_range=(1,2), # N-Gram
                                   norm='l2',
                                   lowercase=False, # Já transformamos tudo em caixa baixa, isso não é mais necessário
                                   stop_words='english', # Para retirar mais possíveis stopwords
                                   max_features=1340000 # Retiramos certa de 4000 features
                                  )
t0 = time()
X_tfidf = vectorizer_tfidf.fit_transform(X)
print('TF-IDF time: %0.3fs' % (time() - t0))

TF-IDF time: 14.490s


#### Hashing + TFIDF
Essa implementação utiliza de _hashs_ para achar o _token_ de uma palavra para a característica numérica. Com essa implementação, alguns benefícios são alcançados:
- Utiliza menos memória para _datasets_ grandes, já que não é necessário armazenar um vocabulário em memória;
- É mais rápido para fazer _pickle_ e _un-pickle_, já que dessa forma não se mantém nenhum estado a não ser o dos parâmetros do construtor;
- Pode ser usado em um fluxo de dados ou _pipelines_ paralelas, já que nenhum estado é computado durante o _fit_.


In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.pipeline import make_pipeline

hasher = HashingVectorizer(stop_words='english', # Para retirar mais possíveis stopwords
                           alternate_sign=False,
                           norm=None,
                           binary=False,
                           n_features=1040000 # Retiramos certa de 5000 features
                          )
vectorizer = make_pipeline(hasher, TfidfTransformer())
t0 = time()
X_hs = vectorizer.fit_transform(X)
print('Hashing + TF-IDF time: %0.3fs' % (time() - t0))

Hashing + TF-IDF time: 3.245s


### Separar dados de treinamento e teste
Agora que temos nossos dados processados númericamente podemos utilizar um algoritimo de classificação para treinar um modelo.   
Utilizaremos 0.8 / 0.2 para o treinamento de nossos modelos.

In [16]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

X_train_bw, X_test_bw, y_train_bw, y_test_bw = train_test_split(X_bow, y, test_size=0.20)
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X_tfidf, y, test_size=0.20)
X_train_hs, X_test_hs, y_train_hs, y_test_hs = train_test_split(X_hs, y, test_size=0.20)

### Treinamento
Para o treinamento de um classificador, chegamos aos seguintes algoritimos:
- Naive Bayes Multinomial
- Stochastic Gradient Descent Classifier
- Support Vector Machine Linear
- Passive Agressive Classifier

Para mais informações sobre tais algoritimos refira-se a [página de conhecimento do grupo](https://github.com/Segmentation-Fault-Machine-Learning/Knowledge/wiki/An%C3%A1lise-dos-algoritmos).

#### Naive Bayes Multinomial

In [17]:
from sklearn.naive_bayes import MultinomialNB
clf_nb = MultinomialNB()

#### Stochastic Gradient Descent Classifier

In [18]:
from sklearn.linear_model import SGDClassifier
clf_sgd = SGDClassifier(alpha=.0001, max_iter=50, penalty="l2")

#### Support Vector Machine Linear

In [19]:
from sklearn.svm import LinearSVC
clf_svm = LinearSVC(penalty='l2', dual=True, tol=1e-3)

#### Passive Agressive Classifier

In [20]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf_pa = PassiveAggressiveClassifier(max_iter=50)

#### Escolha do Classificador

In [22]:
import numpy as np
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, recall_score

classifiers = {
    'NaiveBayesMultinomial': clf_nb,
    'SGDClassifier': clf_sgd,
    'SVMLinear': clf_svm,
    'PassiveAgressive': clf_pa
}
X_s = {
    'BagOfWords': X_bow, 
    'TF-IDF': X_tfidf, 
    'Hashing+TFIDF': X_hs
}
scoring = {'prec_macro': 'precision_macro',
           'rec_micro': make_scorer(recall_score, average='macro')}
t0 = time()
for X in X_s:
    print (X)
    for clf in classifiers:
        scores = cross_validate(classifiers[clf], X_s[X], y, scoring=scoring,
                                cv=10, return_train_score=True)
        print ('\t {} - {}'.format(clf, np.mean(scores['test_prec_macro'])))

print('Execution time %0.3fs' % (time() - t0))

BagOfWords
	 NaiveBayesMultinomial - 0.8893273835151685
	 SGDClassifier - 0.8771647302920214
	 SVMLinear - 0.9015634925106856
	 PassiveAgressive - 0.9018596049848584
TF-IDF
	 NaiveBayesMultinomial - 0.9104694585573714
	 SGDClassifier - 0.9350964504055984
	 SVMLinear - 0.9421406846014694
	 PassiveAgressive - 0.9416056553492668
Hashing+TFIDF
	 NaiveBayesMultinomial - 0.8963245447055203
	 SGDClassifier - 0.9226840317574634
	 SVMLinear - 0.9329969907055279
	 PassiveAgressive - 0.9283832737401811
Execution time 1345.506s


### Refinamento dos hyper-parâmetros
Após a execução dos diferentes algoritmos com as diferentes técnicas de extração de feature, chegamos a conclusão de que o algoritmo **SVM Linear** utilizando do **TFIDF** alcançou os melhores resultados. Portanto, escolhemos essa abordagem para a otimização dos hyper-parâmetros.  
A melhor forma de escolher um bom hyper-parâmetro é pela tentativa e erro de todas as possíveis combinações de valores para tais parâmetros. Na biblioteca do _Scikit-learn_ podemos encontrar a função **GridSeachCV** e a **RandomSearchCV** que facilitam essa tentativa e erro.   
Utilizando do **GridSearch** para um dado modelo, podemos definir um conjunto de parâmetros e valores que queremos testar, então a função _GridSearchCV_ da biblioteca do _Scikit-learn_ cria modelos para todas as combinações possíveis dentro da lista prédefinida de valores dos hyper-parâmetros. Ao final da criação de todos os modelos a melhor combinação é escolhida através da técnica de avaliação _cross-validation_. Com isso podemos notar duas desvantagens:
- Computacionalmente caro. Quanto mais parâmetros se quer testar mais caro computacionalmente será. Considereando 5 parâmetros e assuminfo que tenha 5 valores a serem testados para cada parâmetro e um _cross-validation_ de 3 _k-folds_ (padrão da biblioteca) isso resulta em $5^5 = 3125\ combinações$, $3125 * 3 = 9375\ fit\ de\ modelos$
- Esse método por ser tentativa e erro não encontra os parâmetros ideais para seu problema, porém pode encontrar parâmetros quas perfeitos, desde de que inicialmente utilizemos bons "chutes".

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    #('hash', HashingVectorizer()),
    ('tfidf', TfidfVectorizer()),
    ('clf', LinearSVC()),
])

parameters = {
    #'hash__n_features': (None, 1040000),
    #'hash__norm': (None, 'l1', 'l2'),
    #'hash__ngram_range': ((1, 1), (1, 2)),
    'tfidf__max_features': (None, 1340000),
    'tfidf__lowercase': (False, ),
    #'tfidf__smooth_idf': (True, False),
    #'tfidf__sublinear_tf': (True, False),
    'tfidf__max_df': (0.6, 0.7, 0.8),
    #'tfidf__min_df': (0.7, 0.8, 1),
    'tfidf__ngram_range': ((1, 1), (1, 2)),
    'tfidf__stop_words': (None, 'english'),
    'tfidf__norm': ('l2', ),
    'clf__penalty': ('l2', ),
    #'clf__dual': (True, False),
    'clf__tol': (1e-2, 1e-3, 1e-5),
    'clf__C': (0.5, 0.7, 0.9),
    #'clf__loss': ('hinge', 'squared_hinge'),
    'clf__max_iter': (100, 1000, 2000)
}

grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
t0 = time()
grid_search.fit(X, y)
print("done in %0.3fs\n" % (time() - t0))

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 648 candidates, totalling 1944 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 27.5min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 63.5min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 113.1min
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed: 179.2min
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed: 264.3min
[Parallel(n_jobs=-1)]: Done 1944 out of 1944 | elapsed: 291.1min finished


done in 17498.988s

Best score: 0.931
Best parameters set:
	clf__C: 0.9
	clf__max_iter: 100
	clf__penalty: 'l2'
	clf__tol: 0.001
	tfidf__lowercase: False
	tfidf__max_df: 0.6
	tfidf__max_features: None
	tfidf__ngram_range: (1, 2)
	tfidf__norm: 'l2'
	tfidf__stop_words: None


In [49]:
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib

vectorizer_tfidf1 = TfidfVectorizer(min_df=1,
                                   max_df=0.6, # Retiramos palavras com alta recorrência em documentos, como "From" e "Subject"
                                   ngram_range=(1,2), # N-Gram
                                   norm='l2',
                                   lowercase=False, # Já transformamos tudo em caixa baixa, isso não é mais necessário
                                   stop_words=None, # Para retirar mais possíveis stopwords
                                   max_features=None # Retiramos certa de 4000 features
                                  )
t0 = time()
X_tfidf1 = vectorizer_tfidf1.fit_transform(X)
print('TF-IDF time: %0.3fs' % (time() - t0))
clf_svm1 = LinearSVC(penalty='l2', dual=True, tol=0.001, C=0.9, max_iter=100)

score = cross_validate(clf_svm1, X_tfidf1, y, scoring=scoring,
                                cv=10, return_train_score=True)
print ('\t SVMLinear with hyperparameter tuning - {}'.format(np.mean(score['test_prec_macro'])))

joblib.dump(clf_svm1, '20newgroupsModel.sav')

TF-IDF time: 18.118s
	 SVMLinear with hyperparameter tuning - 0.9419738570273996


['20newgroupsModel.sav']

## Conclusão
Portanto, vimos que para trabalhar com dados textuais a maioria do esforço é empregado no processamento desses dados, para que ao final eles fiquem de forma coerente e númerica para serem utilizados em algoritmos de _machine learning_. Também vimos que utilizando o método de extração de caracteristicas _TF-IDF_ com o classificador _SVM_ com o kernel Linear, obtivemos a melhor predição, com **94%** de acertos.